<a href="https://colab.research.google.com/github/dawikrol/Crypto-analyser/blob/main/Scripts/Solar_energy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###**Lib import:**

In [7]:
import pandas as pd
from google.colab import files
import io
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import os
from sqlalchemy import create_engine

### <a name ='1'></a>**Data uploaded**

Data comes from: https://ec.europa.eu/jrc/en/pvgis.

You can find them in git project director: Data/Solar_energy



In [3]:
uploaded = files.upload()

Saving Limanowski_01.csv to Limanowski_01.csv
Saving Nowosądecki_01.csv to Nowosądecki_01.csv
Saving Nowotarski_01.csv to Nowotarski_01.csv
Saving NowySącz_01.csv to NowySącz_01.csv
Saving Tatrzański_01.csv to Tatrzański_01.csv


###**Data preprocesing**

Preparing raw DataFrame

In [4]:
dataframes = {}

for file in uploaded:
    name, _ = file.split('.')
    dataframes[name] = pd.read_csv(io.BytesIO(uploaded[file]),error_bad_lines=False)

b'Skipping line 34: expected 1 fields, saw 2\n'
b'Skipping line 34: expected 1 fields, saw 2\n'
b'Skipping line 34: expected 1 fields, saw 2\n'
b'Skipping line 34: expected 1 fields, saw 2\n'
b'Skipping line 34: expected 1 fields, saw 2\n'


In [5]:
dataframes[list(dataframes.keys())[0]]  # raw data frame

,Latitude (decimal degrees):\t49.712
0,Longitude (decimal degrees):\t20.406
1,Radiation database:\tPVGIS-SARAH
2,Nominal power of the PV system (c-Si) (kWp):\t1.0
3,System losses(%):\t14.0
4,Fixed slope of modules (deg.) (optimum):\t39
5,Orientation (azimuth) of modules (deg.) (optim...
6,Fixed angle
7,Month\t\tE_d\t\tE_m\t\tH(i)_d\t\tH(i)_m\t\tSD_m
8,1\t\t1.31\t\t40.5\t\t1.49\t\t46.09\t\t13.73
9,2\t\t1.81\t\t50.63\t\t2.07\t\t58.02\t\t13.63


Data clearing

In [6]:
cleared_df = {}

for key in dataframes:
    cl_data = dataframes[key].iloc[7:20]
    cl_data.columns = ['default']
    cl_data = cl_data.default.str.split('\t\t', expand=True)
    cl_data.columns = cl_data.iloc[0]
    cl_data = cl_data.drop(cl_data.index[0])
    cl_data = cl_data.reset_index(drop=True)
    cl_data = cl_data[['Month', 'E_m']]
    cl_data.columns = ['Mounth', 'Mounthly energy production [kWh/m]']
    mounths = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
            'August', 'September', 'Octomber', 'November', 'December']
    cl_data['Mounth'] = mounths
    cl_data.set_index('Mounth', inplace=True)

    cleared_df[key] = cl_data


### Interactive data preview

In [15]:
dropdown_frames = widgets.Dropdown(options = list(cleared_df.keys()))
output = widgets.Output()

def dropdown_frames_eventhandler(change):
    output.clear_output()
    with output:
        display(cleared_df[change.new])

dropdown_frames.observe(dropdown_frames_eventhandler, names='value')

In [16]:
display(dropdown_frames)

Dropdown(options=('Limanowski_01', 'Nowosądecki_01', 'Nowotarski_01', 'NowySącz_01', 'Tatrzański_01'), value='…

In [18]:
display(output)

Output()

Creation of final data

In [22]:
final_data = {
    'January' : [],
    'February': [],
    'March' : [],
    'April' : [],
    'May' : [],
    'June' : [],
    'July' : [],
    'August' : [],
    'September' : [],
    'Octomber' : [],
    'November' : [],
    'December' : []
    }

for key in cleared_df:
    for m in mounths:
        final_data[m].append(float(cleared_df[key].loc[m, 'Mounthly energy production [kWh/m]']))

for key in final_data:
    final_data[key] = round(sum(final_data[key])/len(final_data[key]), 3)


In [23]:
final_df = pd.DataFrame(final_data, index=[1]).T


final_df.columns = ['Mounthly energy production [kWh/m]']
final_df.index.names = ['Mounth']

Final data:

Average value of the potential amount of energy produced from photovoltaics for Zone 1 per year

In [24]:
final_df ['Mounthly energy production [kWh/m]'].sum()

1015.742

### Plot

In [25]:
import plotly.offline as offline
import plotly.graph_objs as go

offline.init_notebook_mode(connected=True)

In [26]:
trace0 = go.Bar(x = final_df.index, y = final_df['Mounthly energy production [kWh/m]'],
                width = 0.5,
                marker = dict(color = 'wheat'))

data = [trace0]

layout = go.Layout(title='Mounthly energy production [kWh/m]',
                  titlefont = dict(family = 'Arial', size = 15),
                  xaxis = dict(tickangle = 30),
                   bargap=1)

fig = go.Figure(data=data, layout=layout)
fig.show(renderer="colab")

### Saving data to files

In [27]:
from google.colab import files


final_df.to_excel('SolarEngergyRegion1.xlsx')
files.download('SolarEngergyRegion1.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Saving to database

In [ ]:
!sudo apt-get install python3-dev default-libmysqlclient-dev
!pip install pymysql
!pip install SQLAlchemy


In [ ]:
MYSQL_HOSTNAME = 'localhost'
MYSQL_USER = os.environ.get('DB_USER')
MYSQL_PASSWORD = os.environ.get('DB_PASS')
MYSQL_DATABASE = 'engineering_data'

connection_string = f'mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOSTNAME}:3306/{MYSQL_DATABASE}'
engine = create_engine(connection_string)

final_df.to_sql('SolarEngergyRegion1', engine)